In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
from supabase import create_client
import pandas as pd
import datetime
from datetime import datetime
import logging
import argparse
import numpy as np
from bs4 import BeautifulSoup
import json
import yfinance as yf

In [2]:
url_supabase = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url_supabase, key)

In [3]:
data_db = supabase.table("sgx_companies").select("*").execute()
data_db = pd.DataFrame(data_db.data)

In [4]:
data_db[data_db["symbol"] == "5NV"]["pcf"]

80   -1.52
Name: pcf, dtype: float64

In [5]:
data_db[data_db["symbol"] == "5OC"]["pcf"]

11    1.47
Name: pcf, dtype: float64

In [6]:
data_sg = pd.read_csv("data_sg.csv")

In [7]:
[col for col in data_db.columns if col not in data_sg.columns]

['historical_earnings',
 'historical_revenue',
 'earnings',
 'forward_dividend',
 'forward_dividend_yield',
 'dividend_ttm']

In [8]:
[col for col in data_sg.columns if col not in data_db.columns]

['dividend', 'dividend_yield']

In [4]:
url = f"https://www.investing.com/equities/koyo-international-ltd-ratios"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)
response

<Response [403]>

In [6]:
monthly = {
            'Name' : 'name', 
            'Symbol' : 'investing_symbol',
            'currency' : 'currency',
            'Sector' : 'sector', 
            'Industry' : 'industry', 
            'Employees' : 'employee_num',
            'Last' : 'close', 
            'ChgPct' : 'percentage_change',
            'FundamentalMarketCap' : 'market_cap', 
            'Volume_x' : 'volume', 
            'FundamentalRatio' : 'pe', 
            'FundamentalRevenue' : 'revenue',
            'EPS' : 'eps',
            'FundamentalBeta' : 'beta', 
            'dividend' : 'dividend',
            'dividend_yield' : 'dividend_yield',
            'TechnicalDay' : 'daily_signal',
            'TechnicalWeek' : 'weekly_signal', 
            'TechnicalMonth' : 'monthly_signal',
            'PerformanceDay' : 'daily_percentage_change', 
            'PerformanceWeek' : 'weekly_percentage_change',
            'PerformanceMonth' : 'monthly_percentage_change', 
            'PerformanceYtd' : 'ytd_percentage_change',
            'PerformanceYear' : 'one_year_percentage_change', 
            'Performance3Year' : 'three_year_percentage_change', 
            'P/E Ratio TTM' : 'pe_ttm',
            'Price to Sales TTM' : 'ps_ttm', 
            'Price to Cash Flow MRQ' : 'pcf', 
            'Price to Free Cash Flow TTM' : 'pcf_ttm', 
            'Price to Book MRQ' : 'pb', 
            '5 Year EPS Growth 5YA' : 'five_year_eps_growth',
            '5 Year Sales Growth 5YA' : 'five_year_sales_growth', 
            '5 Year Capital Spending Growth 5YA' : 'five_year_capital_spending_growth',
            'Asset Turnover TTM' : 'asset_turnover', 
            'Inventory Turnover TTM' : 'inventory_turnover_ttm', 
            'Receivable Turnover TTM' : 'receivable_turnover',
            'Gross margin TTM' : 'gross_margin', 
            'Operating margin TTM' : 'operating_margin', 
            'Net Profit margin TTM' : 'net_profit_margin', 
            'Quick Ratio MRQ' : 'quick_ratio',
            'Current Ratio MRQ' : 'current_ratio', 
            'Total Debt to Equity MRQ' : 'debt_to_equity', 
            'Dividend Yield 5 Year Avg. 5YA' : 'five_year_dividend_average',
            'Dividend Growth Rate ANN' : 'dividend_growth_rate', 
            'Payout Ratio TTM' : 'payout_ratio'
        }
    
daily = {
    'Symbol' : 'investing_symbol',
    'Last' : 'close', 
    'ChgPct' : 'percentage_change',
    'FundamentalMarketCap' : 'market_cap', 
    'Volume' : 'volume', 
    'FundamentalRatio' : 'pe', 
    'FundamentalRevenue' : 'revenue',
    'FundamentalBeta' : 'beta', 
    'TechnicalDay' : 'daily_signal',
    'TechnicalWeek' : 'weekly_signal', 
    'TechnicalMonth' : 'monthly_signal',
    'PerformanceDay' : 'daily_percentage_change', 
    'PerformanceWeek' : 'weekly_percentage_change',
    'PerformanceMonth' : 'monthly_percentage_change', 
    'PerformanceYtd' : 'ytd_percentage_change',
    'PerformanceYear' : 'one_year_percentage_change', 
    'Performance3Year' : 'three_year_percentage_change',
}

In [16]:
monthly_updater_new_col = [col for col in monthly.values() if col not in daily.values()]
not_monthly = ["revenue", 'dividend', 'dividend_yield', 'dividend_ttm','forward_dividend','forward_dividend_yield','net_profit_margin',"operating_margin","gross_margin","quick_ratio","current_ratio","debt_to_equity","payout_ratio","eps"]
yf_daily_col = ["market_cap", "pe", "ps_ttm", "pb", "beta", "volume", "name"]
[col for col in monthly_updater_new_col if col not in not_monthly]

['name',
 'currency',
 'sector',
 'industry',
 'employee_num',
 'pe_ttm',
 'ps_ttm',
 'pcf',
 'pcf_ttm',
 'pb',
 'five_year_eps_growth',
 'five_year_sales_growth',
 'five_year_capital_spending_growth',
 'asset_turnover',
 'inventory_turnover_ttm',
 'receivable_turnover',
 'five_year_dividend_average',
 'dividend_growth_rate']

In [17]:
monthly_col = ["Volume", "Market Cap", "Revenue", "P/E Ratio", "EPS",  "Dividend (Yield)", "P/E Ratio TTM", 
 "Price to Sales TTM", "Price to Cash Flow MRQ", "Price to Free Cash Flow TTM", "Price to Book MRQ",
"5 Year EPS Growth 5YA", "5 Year Sales Growth 5YA", "5 Year Capital Spending Growth 5YA", "Asset Turnover TTM",
"Inventory Turnover TTM", "Receivable Turnover TTM", "Gross margin TTM", "Operating margin TTM", "Net Profit margin TTM",
"Quick Ratio MRQ", "Current Ratio MRQ", "Total Debt to Equity MRQ", "Dividend Yield 5 Year Avg. 5YA", "Dividend Growth Rate ANN",
"Payout Ratio TTM", "Industry", "Sector", "Employees"]
monthly_renamed = [val for key, val in zip(monthly.keys(), monthly.values()) if key in monthly_col and val not in not_monthly and val not in yf_daily_col]
monthly_renamed

['sector',
 'industry',
 'employee_num',
 'pe_ttm',
 'pcf',
 'pcf_ttm',
 'five_year_eps_growth',
 'five_year_sales_growth',
 'five_year_capital_spending_growth',
 'asset_turnover',
 'inventory_turnover_ttm',
 'receivable_turnover',
 'five_year_dividend_average',
 'dividend_growth_rate']

In [18]:
[col for col in data_db.columns if col in monthly_renamed]

['pe_ttm',
 'pcf',
 'pcf_ttm',
 'five_year_eps_growth',
 'five_year_sales_growth',
 'five_year_capital_spending_growth',
 'asset_turnover',
 'inventory_turnover_ttm',
 'receivable_turnover',
 'dividend_growth_rate',
 'sector',
 'employee_num']

# Yf updater

In [4]:
url_currency = 'https://raw.githubusercontent.com/supertypeai/sectors_get_conversion_rate/master/conversion_rate.json'
response = requests.get(url_currency)
data = response.json()

In [5]:
data_prep = data_db.copy()

In [6]:
symbol = yf.Ticker("Z74.SI")
symbol.info

{'address1': 'Singapore Post Centre',
 'address2': 'No. 07-31 10 Eunos Road 8',
 'city': 'Singapore',
 'zip': '408600',
 'country': 'Singapore',
 'phone': '65 6838 3388',
 'fax': '65 6732 8428',
 'website': 'https://www.singtel.com',
 'industry': 'Telecom Services',
 'industryKey': 'telecom-services',
 'industryDisp': 'Telecom Services',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': "Singapore Telecommunications Limited, together with its subsidiaries, provides telecommunication services to consumers and small businesses in Singapore, Australia, China, and internationally. The company operates through Optus, Singtel Singapore, NCS, Digital InfraCo, and Corporate segments. The company provides mobile, equipment sales, fixed voice and data, satellite, ICT and managed services; mobile, fixed voice and data, pay television, content and digital services, ICT as well as equipment sales in Singapor

In [37]:
date_format = "%Y-%m-%d"
# take latest date from database
last_date = datetime.strptime(data_prep["close"][0][-1]["date"], date_format)
last_year = last_date.year
last_month = last_date.month
last_day = last_date.day

# take current date from datetime library
current_date = datetime.now()
current_year = current_date.year
current_month = current_date.month
current_day = current_date.day

# get the list of new dates that not in the db
list_dates = []
if last_year == current_year:
    if last_month == current_month:
        for i in range(last_day + 1, current_day + 1, 1):
            temp_date = datetime(current_year, current_month, i).strftime(date_format)
            list_dates.append(temp_date)
    else:
        max_day = 31
        min_day = 1
        for i in range(current_day + 1, max_day + 1, 1):
            try:
                temp_date = datetime(current_year, last_month, i).strftime(date_format)
                list_dates.append(temp_date)
            except:
                pass
        for i in range(min_day, current_day, 1):
            temp_date = datetime(current_year, current_month, i).strftime(date_format)
            list_dates.append(temp_date)

else:
    max_day = 31
    min_day = 1
    for i in range(current_day + 1, max_day + 1, 1):
        try:
            temp_date = datetime(last_year, last_month, i).strftime(date_format)
            list_dates.append(temp_date)
        except:
            pass
    for i in range(min_day, current_day, 1):
        temp_date = datetime(current_year, current_month, i).strftime(date_format)
        list_dates.append(temp_date)

now = datetime.now()
prev_1month = datetime(now.year, now.month - 1 if now.month > 1 else 12, now.day).strftime(date_format)

new_close = []
for index, row in data_prep.iterrows():
    try:
        # ticker_extension = ".KL" if country == "my" else ".SI"
        # ticker = yf.Ticker(row["symbol"] + ticker_extension)
        ticker = yf.Ticker(row["symbol"] + ".SI")
        currency = row["currency"]
        country_currency = "SGD" # need to add myr
        
        # update data from info yfinance
        data_json = ticker.info
        desired_values = {
            "marketCap" : "market_cap",
            "volume" : "volume",
            "trailingPE" : "pe", 
            "priceToSalesTrailing12Months" : "ps_ttm", 
            "priceToBook" : "pb", 
            "beta" : "beta"
            }
        for key_dv, val_dv in zip(desired_values.keys(), desired_values.values()):
            try:
                if val_dv == "market_cap":
                    if currency != country_currency:
                        rate = float(data[currency][country_currency])
                        temp_val = data_json[key_dv] * rate
                    else:
                        temp_val = data_json[key_dv]
                    data_prep.loc[index, val_dv] = temp_val
                else:
                    data_prep.loc[index, val_dv] = data_json[key_dv]
            except Exception as e:
                """
                if this appear, that means yf don't have the data of the metrics
                so it will be filled by NaN, or we can just still used investing.com values
                for "pe" it will be calculated first with this formula, pe = close/eps_ttm
                """
                if val_dv == "pe":
                    temp_pe = row["close"][-1]["close"]/row["eps"]
                    data_prep.loc[index, "pe"] = temp_pe
                else:
                    data_prep.loc[index, val_dv] = np.nan

        # update data from history yfinance
        yf_data = ticker.history(period="1mo").reset_index()
        close_data = row["close"]
        for i in range(len(yf_data)):
            curr = yf_data.iloc[i]
            curr_date = curr["Date"].strftime(date_format)
            if curr_date in list_dates:
                curr_close =  float(curr["Close"])
                if currency != country_currency:
                    rate = float(data[currency][country_currency])
                    curr_close = float(curr["Close"])*rate
                else:
                    curr_close = float(curr["Close"])
                temp = {
                    "date" : curr_date,
                    "close" : curr_close
                }
                close_data.append(temp)
        close_data = [close for close in close_data if close["date"] > prev_1month]
        new_close.append(close_data)
    except Exception as e:
        symbol = row["symbol"]
        print(f"error in symbol {symbol} : ", e)
        new_close.append(np.nan)
data_prep = data_prep.assign(close = new_close)

In [61]:
def yfinance_updater(data_prep, country):
    date_format = "%Y-%m-%d"
    # take latest date from database
    last_date = datetime.strptime(data_prep["close"][0][-1]["date"], date_format)
    last_year = last_date.year
    last_month = last_date.month
    last_day = last_date.day

    # take current date from datetime library
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    current_day = current_date.day

    # get the list of new dates that not in the db
    list_dates = []
    if last_year == current_year:
        if last_month == current_month:
            for i in range(last_day + 1, current_day + 1, 1):
                temp_date = datetime(current_year, current_month, i).strftime(date_format)
                list_dates.append(temp_date)
        else:
            max_day = 31
            min_day = 1
            for i in range(current_day + 1, max_day + 1, 1):
                try:
                    temp_date = datetime(current_year, last_month, i).strftime(date_format)
                    list_dates.append(temp_date)
                except:
                    pass
            for i in range(min_day, current_day, 1):
                temp_date = datetime(current_year, current_month, i).strftime(date_format)
                list_dates.append(temp_date)

    else:
        max_day = 31
        min_day = 1
        for i in range(current_day + 1, max_day + 1, 1):
            try:
                temp_date = datetime(last_year, last_month, i).strftime(date_format)
                list_dates.append(temp_date)
            except:
                pass
        for i in range(min_day, current_day, 1):
            temp_date = datetime(current_year, current_month, i).strftime(date_format)
            list_dates.append(temp_date)

    now = datetime.now()
    prev_1month = datetime(now.year, now.month - 1 if now.month > 1 else 12, now.day).strftime(date_format)

    new_close = []
    for index, row in data_prep.iterrows():
        try:
            ticker_extension = ".KL" if country == "my" else ".SI"
            ticker = yf.Ticker(row["symbol"] + ticker_extension)
            currency = row["currency"]
            country_currency = "MYR" if country == "my" else "SGD"
            
            # update data from info yfinance
            data_json = ticker.info
            desired_values = {
                "marketCap" : "market_cap",
                "volume" : "volume",
                "trailingPE" : "pe", 
                "priceToSalesTrailing12Months" : "ps_ttm", 
                "priceToBook" : "pb", 
                "beta" : "beta"
                }
            for key_dv, val_dv in zip(desired_values.keys(), desired_values.values()):
                try:
                    if val_dv == "market_cap":
                        if currency != country_currency:
                            rate = float(data[currency][country_currency])
                            temp_val = data_json[key_dv] * rate
                        else:
                            temp_val = data_json[key_dv]
                        data_prep.loc[index, val_dv] = temp_val
                    else:
                        data_prep.loc[index, val_dv] = data_json[key_dv]
                except Exception as e:
                    """
                    if this appear, that means yf don't have the data of the metrics
                    so it will be filled by NaN, or we can just still used investing.com values
                    for "pe" it will be calculated first with this formula, pe = close/eps_ttm
                    """
                    if val_dv == "pe":
                        temp_pe = row["close"][-1]["close"]/row["eps"]
                        data_prep.loc[index, "pe"] = temp_pe
                    else:
                        data_prep.loc[index, val_dv] = np.nan

            # update data from history yfinance
            yf_data = ticker.history(period="1mo").reset_index()
            close_data = row["close"]
            for i in range(len(yf_data)):
                curr = yf_data.iloc[i]
                curr_date = curr["Date"].strftime(date_format)
                if curr_date in list_dates:
                    curr_close =  float(curr["Close"])
                    if currency != country_currency:
                        rate = float(data[currency][country_currency])
                        curr_close = float(curr["Close"])*rate
                    else:
                        curr_close = float(curr["Close"])
                    temp = {
                        "date" : curr_date,
                        "close" : curr_close
                    }
                    close_data.append(temp)
            close_data = [close for close in close_data if close["date"] > prev_1month]
            new_close.append(close_data)
        except Exception as e:
            symbol = row["symbol"]
            print(f"error in symbol {symbol} : ", e)
            new_close.append(np.nan)
    data_prep = data_prep.assign(close = new_close)
    return data_prep

In [62]:
data_final = yfinance_updater(data_db, "sg")

In [55]:
data_db[data_db["currency"] != "SGD"][["symbol", "currency","close", "market_cap", "volume", "pe", "ps_ttm", "pb", "beta", "eps"]]

,symbol,currency,close,market_cap,volume,pe,ps_ttm,pb,beta,eps
49,T41,HKD,"[{'date': '2024-06-10', 'close': 0.2292815877}...",476760000,0,9.357142,0.19,0.24,0.109,-0.03
74,H78,USD,"[{'date': '2024-06-10', 'close': 4.4678299404}...",9574400000,2894000,-12.05,3.86,0.22,0.523,-0.26
83,A05,AUD,"[{'date': '2024-06-10', 'close': 0.2416890097}...",143726960,0,13.500001,NaN,NaN,0.791,0.02
94,T24,USD,"[{'date': '2024-06-10', 'close': 3.4959756381}...",4093600000,72800,13.058824,2.70,3.16,0.698,NaN
146,K6S,USD,"[{'date': '2024-06-10', 'close': 13.5114741689...",33442400000,0,14.983871,NaN,NaN,1.270,0.62
159,CMOU,USD,"[{'date': '2024-06-10', 'close': 0.1876218629}...",186075200,489500,-2.03,0.93,0.19,1.260,-0.06
187,D01,USD,"[{'date': '2024-06-10', 'close': 2.5376194598}...",3427200000,500800,93.5,0.28,2.62,0.000,0.02
315,M04,USD,"[{'date': '2024-06-10', 'close': 2.3081539277}...",2910400000,25800,-5.85,3.92,0.74,0.521,-0.29
349,BTOU,USD,"[{'date': '2024-06-10', 'close': 0.0944858316}...",149804000,280700,-0.29,0.55,0.19,0.965,-0.21
351,NS8U,USD,"[{'date': '2024-06-10', 'close': 0.1687246986}...",1564000000,7366800,Infinity,0.81,0.34,0.866,NaN


In [59]:
rate = float(data["USD"]["SGD"])
temp = yf.Ticker("T24.SI")
yf_data = temp.info
mcap = yf_data["marketCap"]
mcap

273605216

In [63]:
data_final[data_final["currency"] != "SGD"][["symbol", "currency","close", "market_cap", "volume", "pe", "ps_ttm", "pb", "beta", "eps"]]

,symbol,currency,close,market_cap,volume,pe,ps_ttm,pb,beta,eps
49,T41,HKD,"[{'date': '2024-06-10', 'close': 0.2292815877}...",5.895529e+06,50300.0,-0.4325,0.143132,1.086957,0.285,-0.03
74,H78,USD,"[{'date': '2024-06-10', 'close': 4.4678299404}...",9.592177e+09,689900.0,-16.719231,3.852581,0.222284,0.534,-0.26
83,A05,AUD,"[{'date': '2024-06-10', 'close': 0.2416890097}...",1.034879e+08,12000.0,13.500001,0.447179,0.328068,0.921,0.02
94,T24,USD,"[{'date': '2024-06-10', 'close': 3.4959756381}...",3.693670e+08,334800.0,Infinity,0.900847,0.222222,0.907,NaN
146,K6S,USD,"[{'date': '2024-06-10', 'close': 13.5114741689...",3.443557e+10,300.0,14.983871,2.310911,1.430111,1.262,0.62
159,CMOU,USD,"[{'date': '2024-06-10', 'close': 0.1876218629}...",1.889410e+08,672500.0,-3.015,0.928357,0.193642,1.332,-0.06
187,D01,USD,"[{'date': '2024-06-10', 'close': 2.5376194598}...",3.344193e+09,862900.0,91.50001,0.270143,2.527624,0.533,0.02
315,M04,USD,"[{'date': '2024-06-10', 'close': 2.3081539277}...",2.832198e+09,26900.0,-7.727586,3.759048,0.708796,0.534,-0.29
349,BTOU,USD,"[{'date': '2024-06-10', 'close': 0.0944858316}...",1.534957e+08,584600.0,-0.411429,0.546571,0.186589,0.995,-0.21
351,NS8U,USD,"[{'date': '2024-06-10', 'close': 0.1687246986}...",1.517038e+09,1361500.0,Infinity,0.105658,0.043848,0.862,NaN


In [42]:
data_prep[data_prep["currency"] != "SGD"][["symbol", "currency","close", "market_cap", "volume", "pe", "ps_ttm", "pb", "beta"]]

,symbol,currency,close,market_cap,volume,pe,ps_ttm,pb,beta
49,T41,HKD,"[{'date': '2024-06-10', 'close': 0.2292815877}...",5.895529e+06,50300.0,-0.438267,0.143132,1.086957,0.285
74,H78,USD,"[{'date': '2024-06-10', 'close': 4.4678299404}...",9.592177e+09,689900.0,-16.615385,3.852581,0.222284,0.534
83,A05,AUD,"[{'date': '2024-06-10', 'close': 0.2416890097}...",1.034879e+08,12000.0,13.500001,0.447179,0.328068,0.921
94,T24,USD,"[{'date': '2024-06-10', 'close': 3.4959756381}...",3.693670e+08,334800.0,Infinity,0.900847,0.222222,0.907
146,K6S,USD,"[{'date': '2024-06-10', 'close': 13.5114741689...",3.443557e+10,300.0,14.983871,2.310911,1.430111,1.262
159,CMOU,USD,"[{'date': '2024-06-10', 'close': 0.1876218629}...",1.889410e+08,672500.0,-3.015,0.928357,0.193642,1.332
187,D01,USD,"[{'date': '2024-06-10', 'close': 2.5376194598}...",3.344193e+09,862900.0,91.50001,0.270143,2.527624,0.533
315,M04,USD,"[{'date': '2024-06-10', 'close': 2.3081539277}...",2.832198e+09,26900.0,-7.727586,3.759048,0.708796,0.534
349,BTOU,USD,"[{'date': '2024-06-10', 'close': 0.0944858316}...",1.534957e+08,584600.0,-0.417857,0.546571,0.186589,0.995
351,NS8U,USD,"[{'date': '2024-06-10', 'close': 0.1687246986}...",1.517038e+09,1361500.0,Infinity,0.105658,0.043848,0.862


In [24]:
data_prep["close"].loc[0]

[{'date': '2024-06-10', 'close': 0.0350000001},
 {'date': '2024-06-11', 'close': 0.0350000001},
 {'date': '2024-06-12', 'close': 0.0350000001},
 {'date': '2024-06-13', 'close': 0.0350000001},
 {'date': '2024-06-14', 'close': 0.0350000001},
 {'date': '2024-06-18', 'close': 0.0350000001},
 {'date': '2024-06-19', 'close': 0.0350000001},
 {'date': '2024-06-20', 'close': 0.0350000001},
 {'date': '2024-06-21', 'close': 0.0350000001},
 {'date': '2024-06-24', 'close': 0.0350000001},
 {'date': '2024-06-25', 'close': 0.0350000001},
 {'date': '2024-06-26', 'close': 0.0350000001},
 {'date': '2024-06-27', 'close': 0.03500000014901161},
 {'date': '2024-06-28', 'close': 0.03500000014901161},
 {'date': '2024-07-01', 'close': 0.03500000014901161},
 {'date': '2024-07-02', 'close': 0.03500000014901161},
 {'date': '2024-07-03', 'close': 0.03500000014901161},
 {'date': '2024-07-04', 'close': 0.03500000014901161},
 {'date': '2024-07-05', 'close': 0.03500000014901161},
 {'date': '2024-07-08', 'close': 0.0350

In [54]:
records = data_db.replace({np.nan: None}).to_dict("records")
try:
    supabase.table("sgx_companies").upsert(records, returning='minimal').execute()
    print("Upsert operation successful.")
except Exception as e:
    print(f"Error during upsert operation: {e}")

Upsert operation successful.


# Janji bakal rapiin nanti

yf data updater and employee updater function to upsert it manually

In [17]:
def yf_data_updater(data_final, country):
    data_dict = {
        "close" : [],
        "market_cap" : [],
        "volume" : [],
        "pe" : [],
        "pb" : [],
        "ps" : [],
        "pcf" : [],
        "beta" : []
    }
    close_list = []
    for index, row in data_final.iterrows():
        try:
            now = datetime.now()
            prev_1month = datetime(now.year, now.month - 1 if now.month > 1 else 12, now.day).strftime("%Y-%m-%d")
            ticker_extension = ".KL" if country == "my" else ".SI"
            ticker = yf.Ticker(row["symbol"] + ticker_extension)
            yf_data = ticker.history(period="1mo").reset_index()
            curr = yf_data.iloc[-1]
            curr_date = curr["Date"].strftime("%Y-%m-%d")
            currency = row["currency"]
            data_final.loc[index, "volume"] = curr["Volume"]
            country_currency = "MYR" if country == "my" else "SGD"
            if currency != country_currency:
                rate = float(data[currency][country_currency])
                market_cap = row["market_cap"]*rate
                data_final.loc[index, "market_cap"] = market_cap
                revenue = row["revenue"]*rate
                data_final.loc[index, "revenue"] = revenue
                dividend = row["dividend"]*rate
                data_final.loc[index, "dividend"] = dividend
                curr_close = float(curr["Close"])*rate
            else:
                curr_close = float(curr["Close"])
            current_data = {
                "date" : curr_date,
                "close" : curr_close
            }
            close = [data for data in row["close"] if data["date"] > prev_1month]
            if current_data["date"] > close[-1]["date"]:
                close.append(current_data)
            close_list.append(close)
        except Exception as e:
            symbol = row["symbol"]
            logging.error(f"error in {symbol}: ", e)
            close_list.append(list())
    data_final = data_final.assign(close = close_list)
    return data_final

def employee_updater(data_final, country):
    # getting the employee_num from sgx web
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    data_dict = {
        "investing_symbol" : [],
        "status" : [],
        "employee_num_sgx" : []
    }
    special_case = {
    'SRTA.SI' : 'STA.BK',
    'CERG.SI' : '1130.HK',
    'CTDM_p.SI' : 'CTDM.SI',
    'TIAN.SI' : '600329.SS',
    'UOAL.SI' : 'UOS.AX',
    'WLAR.SI' : '0854.HK',
    'IHHH.SI' : 'IHHH.KL',
    'TOPG.SI' : 'TPGC.KL',
    'AVJH.SI' : 'AVJ.AX',
    'MYSC.SI' : 'MSCB.KL',
    'SHNG.SI' : '0069.HK',
    'PRTL.SI' : 'PRU.L',
    'AMTI.SI' : 'AMTD.K',
    'STELy.SI' : 'STEL.SI',
    'COUA.SI' : '1145.HK',
    'SRIT.SI' : 'STGT.BK',
    'NIOI.SI' : 'NIO',
    'EMPE.SI' : 'EMI.PS',
    'YUNN.SI' : '1298.HK',
    'CKFC.SI' : '0834.HK',
    'COMB.SI' : '2342.HK'
    }
    for symbol in data_db["investing_symbol"].tolist():
        data_dict["investing_symbol"].append(symbol)
        ticker_extension = ".KL" if country == "my" else ".SI"
        symbol = symbol + ticker_extension
        if symbol in special_case.keys():
            for key, value in zip(special_case.keys(), special_case.values()):
                if symbol == key:
                    url = f"https://api.sgx.com/companygeneralinformation/v1.0/countryCode/SG/ricCode/{value}?lang=en-US&params=companyDescription%2CstreetAddress1%2CstreetAddress2%2CstreetAddress3%2Ccity%2Cstate%2CpostalCode%2Ccountry%2Cemail%2Cwebsite%2CincorporatedDate%2CincorporatedCountry%2CpublicDate%2CnoOfEmployees%2CnoOfEmployeesLastUpdated"    
        else:
            url = f"https://api.sgx.com/companygeneralinformation/v1.0/countryCode/SG/ricCode/{symbol}?lang=en-US&params=companyDescription%2CstreetAddress1%2CstreetAddress2%2CstreetAddress3%2Ccity%2Cstate%2CpostalCode%2Ccountry%2Cemail%2Cwebsite%2CincorporatedDate%2CincorporatedCountry%2CpublicDate%2CnoOfEmployees%2CnoOfEmployeesLastUpdated"
        response = requests.get(url)
        if response.status_code == 200:
            data_dict["status"].append(response.status_code)
            try:
                employee_num_sgx = response.json()["data"][0]["noOfEmployees"]
            except:
                employee_num_sgx = np.nan
            data_dict["employee_num_sgx"].append(employee_num_sgx)
        else:
            data_dict["status"].append(response.status_code)
            data_dict["employee_num_sgx"].append(np.nan)
    employee_sgx = pd.DataFrame(data_dict).drop("status", axis = 1)
    employee_num_all = pd.merge(data_final, employee_sgx, on = "investing_symbol", how = "left")
    new_en = []
    for en_investing, en_sgx in zip(employee_num_all["employee_num"].tolist(), employee_num_all["employee_num_sgx"].tolist()):
        if en_sgx > 0:
            new_en.append(en_sgx)
        else:
            if en_investing > 0:
                new_en.append(en_investing)
            else:
                new_en.append(np.nan)
    data_final = data_final.assign(employee_num = new_en)
    return data_final

In [17]:
def employee_updater(data_final, country):
    # getting the employee_num from sgx web
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    iv_data_dict = {
        "investing_symbol" : [],
        "status" : [],
        "employee_num_sgx" : []
    }
    yf_data_dict = {
        "investing_symbol" : [],
        "employee_num" : []
    }
    special_case = {
    'SRTA.SI' : 'STA.BK',
    'CERG.SI' : '1130.HK',
    'CTDM_p.SI' : 'CTDM.SI',
    'TIAN.SI' : '600329.SS',
    'UOAL.SI' : 'UOS.AX',
    'WLAR.SI' : '0854.HK',
    'IHHH.SI' : 'IHHH.KL',
    'TOPG.SI' : 'TPGC.KL',
    'AVJH.SI' : 'AVJ.AX',
    'MYSC.SI' : 'MSCB.KL',
    'SHNG.SI' : '0069.HK',
    'PRTL.SI' : 'PRU.L',
    'AMTI.SI' : 'AMTD.K',
    'STELy.SI' : 'STEL.SI',
    'COUA.SI' : '1145.HK',
    'SRIT.SI' : 'STGT.BK',
    'NIOI.SI' : 'NIO',
    'EMPE.SI' : 'EMI.PS',
    'YUNN.SI' : '1298.HK',
    'CKFC.SI' : '0834.HK',
    'COMB.SI' : '2342.HK'
    }
    for iv_symbol in data_final["investing_symbol"].tolist():
        iv_data_dict["investing_symbol"].append(iv_symbol)
        ticker_extension = ".KL" if country == "my" else ".SI"
        iv_symbol = iv_symbol + ticker_extension
        if iv_symbol in special_case.keys():
            for key, value in zip(special_case.keys(), special_case.values()):
                if iv_symbol == key:
                    url = f"https://api.sgx.com/companygeneralinformation/v1.0/countryCode/SG/ricCode/{value}?lang=en-US&params=companyDescription%2CstreetAddress1%2CstreetAddress2%2CstreetAddress3%2Ccity%2Cstate%2CpostalCode%2Ccountry%2Cemail%2Cwebsite%2CincorporatedDate%2CincorporatedCountry%2CpublicDate%2CnoOfEmployees%2CnoOfEmployeesLastUpdated"    
        else:
            url = f"https://api.sgx.com/companygeneralinformation/v1.0/countryCode/SG/ricCode/{iv_symbol}?lang=en-US&params=companyDescription%2CstreetAddress1%2CstreetAddress2%2CstreetAddress3%2Ccity%2Cstate%2CpostalCode%2Ccountry%2Cemail%2Cwebsite%2CincorporatedDate%2CincorporatedCountry%2CpublicDate%2CnoOfEmployees%2CnoOfEmployeesLastUpdated"
        response = requests.get(url)
        if response.status_code == 200:
            iv_data_dict["status"].append(response.status_code)
            try:
                employee_num_sgx = response.json()["data"][0]["noOfEmployees"]
            except:
                employee_num_sgx = np.nan
            iv_data_dict["employee_num_sgx"].append(employee_num_sgx)
        else:
            iv_data_dict["status"].append(response.status_code)
            iv_data_dict["employee_num_sgx"].append(np.nan)
    for iv_symbol, yf_symbol in zip(data_final["investing_symbol"].tolist(), data_final["symbol"].tolist()):
        yf_data_dict["investing_symbol"].append(iv_symbol)
        try:
            temp = yf.Ticker(yf_symbol + ".SI")
            employee = temp.info["fullTimeEmployees"]
            yf_data_dict["employee_num"].append(employee)
        except:
            yf_data_dict["employee_num"].append(np.nan)
    employee_sgx = pd.DataFrame(iv_data_dict).drop("status", axis = 1)
    employee_yf = pd.DataFrame(yf_data_dict)
    new_en = []
    for en_yf, en_sgx in zip(employee_yf["employee_num"].tolist(), employee_sgx["employee_num_sgx"].tolist()):
        if en_sgx > 0:
            new_en.append(en_sgx)
        else:
            if en_yf > 0:
                new_en.append(en_yf)
            else:
                new_en.append(np.nan)
    data_final = data_final.assign(employee_num = new_en)
    return data_final

In [18]:
data_final = employee_updater(data_db, "sg")

In [12]:
yf

,investing_symbol,employee_num
0,STEG,22494.0
1,BEST,NaN
2,CDWH,571.0
3,CTDM,8083.0
4,ANHL,52.0
...,...,...
589,EVER,NaN
590,SHEF,NaN
591,WINK,725.0
592,LMSC,NaN


In [11]:
sgx

,investing_symbol,employee_num_sgx
0,STEG,25734.0
1,BEST,58.0
2,CDWH,383.0
3,CTDM,8083.0
4,ANHL,138.0
...,...,...
589,EVER,0.0
590,SHEF,0.0
591,WINK,734.0
592,LMSC,0.0


In [76]:
records = data_final.replace({np.nan: None}).to_dict("records")
try:
    supabase.table("sgx_companies").upsert(records, returning='minimal').execute()
    print("Upsert operation successful.")
except Exception as e:
    print(f"Error during upsert operation: {e}")

Upsert operation successful.


Normal function, just use this for testing

In [30]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import urllib.request
proxy = os.environ.get("PROXY")
# from scraper import scrap_function
# from combiner import combine_data
import time
from multiprocessing import Process

proxy_support = urllib.request.ProxyHandler({'http': proxy,'https': proxy})
opener = urllib.request.build_opener(proxy_support)
urllib.request.install_opener(opener)

In [18]:
def clean_daily_foreign_data(foreign_daily_data):
    """
    SGX/KLSE Daily Data Fetching Cleansing.

    Parameters:
    - foreign_daily_data: dataframe, KLSE/SGX daily data from investing.com api hit

    Returns:
    - foreign_daily_data: dataframe, Cleaned KLSE/SGX daily data
    """

    # Replace '-' data with ''
    foreign_daily_data = foreign_daily_data.replace('-',np.nan)

    # Remove percentage and change data to decimal
    for i in ["daily",'weekly','monthly','ytd','one_year','three_year']:
        foreign_daily_data[f"{i}_percentage_change"] = foreign_daily_data[f"{i}_percentage_change"]/100 

    # Rename columns
    foreign_daily_data.rename(columns={"daily_percentage_change":"change_1d", "weekly_percentage_change":'change_7d', 
                        "monthly_percentage_change":"change_1m", 
                        "ytd_percentage_change":"change_ytd",
                        "one_year_percentage_change":"change_1y",
                        "three_year_percentage_change":"change_3y"}, inplace=True)

    # Delete redundant percentage change columns
    foreign_daily_data.drop("percentage_change",axis=1, inplace = True)

    # Change data type to float
    float_columns = ['close', 'market_cap', 'volume','pe', 'revenue', 'beta','change_1d',
       'change_7d', 'change_1m', 'change_ytd', 'change_1y', 'change_3y',]

    foreign_daily_data[float_columns] = foreign_daily_data[float_columns].applymap(lambda x:float(str(x).replace(',', '')))

    return foreign_daily_data

def clean_periodic_foreign_data(foreign_periodic_data, foreign_sectors):
    """
    SGX/KLSE Periodic Data Fetching Cleansing.

    Parameters:
    - foreign_periodic_data: dataframe, periodic data from investing.com data scraping using request
    - foreign_sectors: dataframe, KLSE/SGX sectors mapping to IDX sectors

    Returns:
    - foreign_periodic_data: dataframe, Cleaned KLSE/SGX periodic data
    """

    # Replace '-' data with ''
    foreign_periodic_data = foreign_periodic_data.replace('-',np.nan)

    foreign_periodic_data['dividend_yield'] = foreign_periodic_data['dividend_yield'].apply(lambda x: float(x.strip('%')) / 100 if pd.notnull(x) else np.nan)

    for i in ["gross_margin","operating_margin",'net_profit_margin',"debt_to_equity","five_year_dividend_average",'dividend_growth_rate',"payout_ratio","five_year_eps_growth","five_year_sales_growth","five_year_capital_spending_growth"]:
        foreign_periodic_data[i] = foreign_periodic_data[i].apply(lambda x: float(x.replace('%', '').replace(',', '')) / 100 if pd.notnull(x) else np.nan)

    foreign_periodic_data.rename(columns={"five_year_dividend_average":"dividend_yield_5y_avg"}, inplace=True) 
    
    float_columns = ['eps', 'dividend', 'dividend_yield', 'pe_ttm', 'ps_ttm', 'pcf', 'pcf_ttm', 'pb', 'five_year_eps_growth',
       'five_year_sales_growth', 'five_year_capital_spending_growth',
       'asset_turnover', 'inventory turnover (ttm)', 'receivable_turnover',
       'gross_margin', 'operating_margin', 'net_profit_margin', 'quick_ratio',
       'current_ratio', 'debt_to_equity', 'dividend_yield_5y_avg',
       'dividend_growth_rate', 'payout_ratio']

    foreign_periodic_data[float_columns] = foreign_periodic_data[float_columns].applymap(lambda x:float(str(x).replace(',', '')))

    foreign_periodic_data = foreign_periodic_data.merge(foreign_sectors, on = ["sector",'industry']).drop(["sector",'industry'], axis=1).rename(columns={"sectors_id":"sector","sub_sector_id":"sub_sector"})

    return foreign_periodic_data

In [26]:
def GetGeneralData(country):
    if country == "sg":
        url ="https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=36&filter-domain=&page=0&page-size=1000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    elif country == "my":
        url = "https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=42&filter-domain=&page=0&page-size=2000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for i in range(20):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("mashook")
            json_data = response.json()
            data = pd.DataFrame(json_data["data"])
            break
        else:
            continue
    return data

In [31]:
def GetGeneralData(country):
    if country == "sg":
        url ="https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=36&filter-domain=&page=0&page-size=1000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    elif country == "my":
        url = "https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=42&filter-domain=&page=0&page-size=2000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    data_from_api = None
    with urllib.request.urlopen(url) as response:
        html = response.read()

    data_from_api = json.loads(html)
    data_from_api = pd.DataFrame(data_from_api["data"])

    # for i in range(10):
    #     response = requests.get(url, headers=headers)

    #     if response.status_code == 200:
    #         json_data = response.json()
    #         data_from_api = pd.DataFrame(json_data["data"])
    #         break
    return data_from_api

In [20]:
def GetAdditionalData(links):
    data_list = []
    failed_links = {
        "links" : [],
        "page" : []
    }
    for link in links[:5]:
        try:
            data_dict = {
                "Url" : link
            }
            # Page Overview
            url = f"https://www.investing.com{link}"
            response = requests.get(url)
            if response.status_code == 200:
                html_content = response.text
                soup = BeautifulSoup(html_content, "html.parser")
                close = soup.find(class_ = "text-5xl/9 font-bold text-[#232526] md:text-[42px] md:leading-[60px]").get_text()
                change_percent = soup.find('span', {'data-test': 'instrument-price-change-percent'}).get_text().replace("(", "").replace(")", "")
                currency = soup.find(class_ = "ml-1.5 font-bold").get_text()
                values = soup.find_all(class_ = "flex flex-wrap items-center justify-between border-t border-t-[#e6e9eb] pt-2.5 sm:pb-2.5 pb-2.5")
                expected_values = ["Volume", "Market Cap", "Revenue", "P/E Ratio", "EPS",  "Dividend (Yield)"]
                data_dict["close"] = close
                data_dict["change_percent"] = change_percent
                data_dict["currency"] = currency
                for value in values:
                    value = value.get_text()
                    for expected_value in expected_values:
                        if expected_value in value:
                            if expected_value == "Dividend (Yield)":
                                value = value.replace(expected_value, "")
                                try:
                                    dividend, yields = value.split("(")
                                    data_dict["dividend"] = dividend
                                    data_dict["dividend_yield"] = yields.replace(")", "")
                                except:
                                    data_dict["dividend"] = "-"
                                    data_dict["dividend_yield"] = "-"
                            else:
                                data_dict[expected_value] = value.replace(expected_value, "")
                company_profile = soup.find(class_ = "mt-6 font-semibold md:mt-0")
                desired_infos = ["Industry", "Sector"]
                for info in company_profile:
                    info = info.get_text()
                    for desired_info in desired_infos:
                        if desired_info in info:
                            data_dict[desired_info] = info.replace(desired_info, "")
            else:
                failed_links["links"].append(link)
                failed_links["page"].append("overview")
                logging.error(f"error at overview page with link: {link}")
                print(f"error at overview page with link: {link}")
                
            # Page Ratios
            url = f"https://www.investing.com{link}-ratios"
            response = requests.get(url)
            if response.status_code == 200:
                html_content= response.text
                soup = BeautifulSoup(html_content, "html.parser")
                general_infos = soup.find("div", class_ = "right general-info").find_all("div")
                for general_info in general_infos:
                    val = "S/N:"
                    if val in general_info.get_text():
                        data_dict["yfinance_ticker"] = general_info.find("span", class_ = "elp").get_text()
                values = soup.find_all("tr")
                expected_values = [
                    "P/E Ratio TTM", "Price to Sales TTM", "Price to Cash Flow MRQ", "Price to Free Cash Flow TTM", "Price to Book MRQ",
                    "5 Year EPS Growth 5YA", "5 Year Sales Growth 5YA", "5 Year Capital Spending Growth 5YA", "Asset Turnover TTM",
                    "Inventory Turnover TTM", "Receivable Turnover TTM", "Gross margin TTM", "Operating margin TTM", "Net Profit margin TTM",
                    "Quick Ratio MRQ", "Current Ratio MRQ", "Total Debt to Equity MRQ", "Dividend Yield 5 Year Avg. 5YA", "Dividend Growth Rate ANN",
                    "Payout Ratio TTM"
                ]
                for value in values:
                    temp = value.get_text()
                    for expected_value in expected_values:
                        if expected_value in temp:
                            metric = value.find_all("td")[1].get_text()
                            data_dict[expected_value] = metric
            else:
                failed_links["links"].append(link)
                failed_links["page"].append("ratios")
                logging.error(f"error at ratios page with link: {link}")
                print(f"error at ratios page with link: {link}")
            data_list.append(data_dict)
        except Exception as e:
            logging.error(f"error in {link}: ", e)
            failed_links["links"].append(link)
            failed_links["page"].append("all")
    extension = pd.DataFrame(data_list)
    return extension, failed_links

In [33]:
country = "sg"
data_general = GetGeneralData(country)
links = data_general["Url"].tolist()
extension, failed_links = GetAdditionalData(links)
data_full = pd.merge(data_general, extension, on = "Url", how = "inner")
# Retry the failed links
n_try = 0
if len(failed_links["links"]) != 0:
    failed_links["links"] = [link.split("?")[0] if "?" in link else link for link in failed_links["links"]]
    while len(failed_links["links"]) != 0 or n_try < 10:
        if len(failed_links["links"]) == 0:
            break
        new_extension, failed_links = GetAdditionalData(failed_links["links"])
        n_try += 1
    remaining = data_general[data_general["Url"].isin(failed_links["links"])]
    remaining = remaining.assign(Url = [link.split("?")[0] if "?" in link else link for link in failed_links["links"]])
    updated_extension = pd.merge(remaining, new_extension, on = "Url", how = "inner")
    data_final = pd.concat([data_full[~data_full["Url"].isin(failed_links["links"])], updated_extension])
else:
    data_final = data_full.copy()
data_final = rename_and_convert(data_final, "monthly")
data_final = clean_daily_foreign_data(data_final)
sg_sectors = pd.read_csv("sectors_mapping/sectors_sg.csv", sep = ";")
data_final = clean_periodic_foreign_data(data_final, sg_sectors)
db = "sgx_companies"
data_db = supabase.table(db).select("*").execute()
data_db = pd.DataFrame(data_db.data)
data_final = pd.merge(data_final, data_db[["investing_symbol", "symbol", "close"]], on = "investing_symbol", how = "inner")
data_final = yf_data_updater(data_final, country).drop(["revenue", 'dividend', 'dividend_yield'], axis = 1)
data_final = employee_updater(data_final, country)

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [23]:
def convert_to_number(x):
    if isinstance(x, str):
        if 'T' in x:
            return float(x.replace('T', '')) * 1e12
        elif 'B' in x:
            return float(x.replace('B', '')) * 1e9
        elif 'M' in x:
            return float(x.replace('M', '')) * 1e6
        elif 'K' in x:
            return float(x.replace('K', '')) * 1e3
        else:
            try:
                return float(x.replace(',', ''))
            except ValueError:
                return np.nan
    elif isinstance(x, (int, float)):
        return x
    else:
        return np.nan

def rename_and_convert(data, period):
    if period == "monthly":
        rename_cols = {
            'Name' : 'name', 
            'Symbol' : 'symbol',
            'currency' : 'currency',
            'Sector' : 'sector', 
            'Industry' : 'industry', 
            'Last' : 'close', 
            'ChgPct' : 'percentage_change',
            'FundamentalMarketCap' : 'market_cap', 
            'Volume_x' : 'volume', 
            'FundamentalRatio' : 'pe', 
            'FundamentalRevenue' : 'revenue',
            'EPS' : 'eps',
            'FundamentalBeta' : 'beta', 
            'dividend' : 'dividend',
            'dividend_yield' : 'dividend_yield',
            'TechnicalDay' : 'daily_signal',
            'TechnicalWeek' : 'weekly_signal', 
            'TechnicalMonth' : 'monthly_signal',
            'PerformanceDay' : 'daily_percentage_change', 
            'PerformanceWeek' : 'weekly_percentage_change',
            'PerformanceMonth' : 'monthly_percentage_change', 
            'PerformanceYtd' : 'ytd_percentage_change',
            'PerformanceYear' : 'one_year_percentage_change', 
            'Performance3Year' : 'three_year_percentage_change', 
            'P/E Ratio TTM' : 'pe_ttm',
            'Price to Sales TTM' : 'ps_ttm', 
            'Price to Cash Flow MRQ' : 'pcf', 
            'Price to Free Cash Flow TTM' : 'pcf_ttm', 
            'Price to Book MRQ' : 'pb', 
            '5 Year EPS Growth 5YA' : 'five_year_eps_growth',
            '5 Year Sales Growth 5YA' : 'five_year_sales_growth', 
            '5 Year Capital Spending Growth 5YA' : 'five_year_capital_spending_growth',
            'Asset Turnover TTM' : 'asset_turnover', 
            'Inventory Turnover TTM' : 'inventory_turnover_ttm', 
            'Receivable Turnover TTM' : 'receivable_turnover',
            'Gross margin TTM' : 'gross_margin', 
            'Operating margin TTM' : 'operating_margin', 
            'Net Profit margin TTM' : 'net_profit_margin', 
            'Quick Ratio MRQ' : 'quick_ratio',
            'Current Ratio MRQ' : 'current_ratio', 
            'Total Debt to Equity MRQ' : 'debt_to_equity', 
            'Dividend Yield 5 Year Avg. 5YA' : 'five_year_dividend_average',
            'Dividend Growth Rate ANN' : 'dividend_growth_rate', 
            'Payout Ratio TTM' : 'payout_ratio'
        }
        cleaned_data = data[rename_cols.keys()].rename(rename_cols, axis = 1)

        cleaned_data.replace(['-', 'N/A'], np.nan, inplace=True)
        cleaned_data['revenue'] = cleaned_data['revenue'].apply(convert_to_number)
        cleaned_data['market_cap'] = cleaned_data['market_cap'].apply(convert_to_number)
        return cleaned_data
    elif period == "daily":
        rename_cols = {
            'Symbol' : 'investing_symbol',
            'Last' : 'close', 
            'ChgPct' : 'percentage_change',
            'FundamentalMarketCap' : 'market_cap', 
            'Volume' : 'volume', 
            'FundamentalRatio' : 'pe', 
            'FundamentalRevenue' : 'revenue',
            'FundamentalBeta' : 'beta', 
            'TechnicalDay' : 'daily_signal',
            'TechnicalWeek' : 'weekly_signal', 
            'TechnicalMonth' : 'monthly_signal',
            'PerformanceDay' : 'daily_percentage_change', 
            'PerformanceWeek' : 'weekly_percentage_change',
            'PerformanceMonth' : 'monthly_percentage_change', 
            'PerformanceYtd' : 'ytd_percentage_change',
            'PerformanceYear' : 'one_year_percentage_change', 
            'Performance3Year' : 'three_year_percentage_change',
        }
        cleaned_data = data[rename_cols.keys()].rename(rename_cols, axis = 1)

        cleaned_data.replace(['-', 'N/A'], np.nan, inplace=True)
        cleaned_data['revenue'] = cleaned_data['revenue'].apply(convert_to_number)
        cleaned_data['market_cap'] = cleaned_data['market_cap'].apply(convert_to_number)
        return cleaned_data

In [24]:
data_general = GetGeneralData("my")
data_general = rename_and_convert(data_general, "daily")
data_general = clean_daily_foreign_data(data_general).drop("close", axis = 1)
data_db = supabase.table("klse_companies").select("*").execute()
data_db = pd.DataFrame(data_db.data)
drop_cols = ['market_cap', 'volume', 'pe',
'revenue', 'beta', 'daily_signal', 'weekly_signal',
'monthly_signal', 'change_1d', 'change_7d', 'change_1m',
'change_ytd', 'change_1y', 'change_3y']
data_db.drop(drop_cols, axis = 1, inplace = True)
data_final = pd.merge(data_general, data_db, on = "investing_symbol", how = "inner")

C:\Users\asus\AppData\Local\Temp\ipykernel_6112\1458753836.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  foreign_daily_data[float_columns] = foreign_daily_data[float_columns].applymap(lambda x:float(str(x).replace(',', '')))


In [25]:
url_currency = 'https://raw.githubusercontent.com/supertypeai/sectors_get_conversion_rate/master/conversion_rate.json'
response = requests.get(url_currency)
data = response.json()

In [14]:
temp = yf.Ticker("Z74.SI")
temp.info
# shares_outstanding = temp.info["sharesOutstanding"]*temp.info["previousClose"]

{'address1': 'Singapore Post Centre',
 'address2': 'No. 07-31 10 Eunos Road 8',
 'city': 'Singapore',
 'zip': '408600',
 'country': 'Singapore',
 'phone': '65 6838 3388',
 'fax': '65 6732 8428',
 'website': 'https://www.singtel.com',
 'industry': 'Telecom Services',
 'industryKey': 'telecom-services',
 'industryDisp': 'Telecom Services',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': "Singapore Telecommunications Limited, together with its subsidiaries, provides telecommunication services to consumers and small businesses in Singapore, Australia, China, and internationally. The company operates through Optus, Singtel Singapore, NCS, Digital InfraCo, and Corporate segments. The company provides mobile, equipment sales, fixed voice and data, satellite, ICT and managed services; mobile, fixed voice and data, pay television, content and digital services, ICT as well as equipment sales in Singapor

In [4]:
data_list = []
for symbol in data_db["symbol"] + ".SI":
    data_dict = {}
    temp = yf.Ticker(symbol)
    data_json = temp.info
    desired_values = ["previousClose", "marketCap", "volume", "trailingPE", "priceToSalesTrailing12Months", "priceToBook", "beta"]
    for dv in desired_values:
        try:
            data_dict[dv] = data_json[dv]
        except:
            data_dict[dv] = np.nan
    data_list.append(data_dict)

In [5]:
daily_data = pd.DataFrame(data_list).assign(symbol = data_db["symbol"].tolist())

In [8]:
data_db[["symbol", "close", "market_cap", "volume", "pe", "ps_ttm", "pb", "pcf", "beta"]].isnull().sum()

symbol         0
close          0
market_cap     0
volume         0
pe             0
ps_ttm        45
pb            46
pcf           64
beta           0
dtype: int64

In [7]:
daily_data.isnull().sum()

previousClose                     1
marketCap                         3
volume                            1
trailingPE                      238
priceToSalesTrailing12Months     12
priceToBook                      40
beta                             21
symbol                            0
dtype: int64

In [35]:
def yf_data_update(data_final):
    close_list = []
    for index, row in data_final.iterrows():
        now = datetime.now()
        prev_1month = datetime(now.year, now.month - 1 if now.month > 1 else 12, now.day).strftime("%Y-%m-%d")
        ticker = yf.Ticker(row["symbol"] + ".SI")
        yf_data = ticker.history(period="1mo").reset_index()
        curr = yf_data.iloc[-1]
        curr_date = curr["Date"].strftime("%Y-%m-%d")
        currency = row["currency"]
        data_final.loc[index, "volume"] = curr["Volume"]
        # try:
        #     if currency != "SGD":
        #         rate = float(data[currency]['SGD'])
        #         data_final.loc[index, "market_cap"] = ticker.info["marketCap"]*rate
        #     else:
        #         data_final.loc[index, "market_cap"] = ticker.info["marketCap"]
        # except:
        #     symbol = row["symbol"]
        #     print(f"error at {symbol} have no market cap")
        #     data_final.loc[index, "market_cap"] = np.nan
        
        if currency != "SGD": # add for MYR
            rate = float(data[currency]['SGD'])
            market_cap = row["market_cap"]*rate
            data_final.loc[index, "market_cap"] = market_cap
            curr_close = float(curr["Close"])*rate
        else:
            curr_close = float(curr["Close"])
        current_data = {
            "date" : curr_date,
            "close" : curr_close
        }
        close = [data for data in row["close"] if data["date"] > prev_1month]
        if current_data["date"] > close[-1]["date"]:
            close.append(current_data)
        close_list.append(close)
    data_final = data_final.assign(close = close_list)
    return data_final

In [36]:
yf_data_update(data_final)

,investing_symbol,market_cap,volume,pe,revenue,beta,daily_signal,weekly_signal,monthly_signal,change_1d,...,quick_ratio,current_ratio,debt_to_equity,dividend_yield_5y_avg,dividend_growth_rate,payout_ratio,sector,sub_sector,symbol,close
0,KPLM,1.178000e+10,2669100.0,2.86,6.970000e+09,0.755,strong_sell,strong_sell,buy,0.0015,...,NaN,1.04,NaN,NaN,NaN,0.1454,Consumer Cyclicals,Apparel & Luxury Goods,BN4,"[{'date': '2024-05-29', 'close': 6.7199997902}..."
1,HKLD,9.751200e+09,1362900.0,-12.24,1.840000e+09,0.523,strong_sell,neutral,strong_sell,0.0156,...,0.46,1.70,0.2056,0.0561,NaN,-0.8350,Properties & Real Estate,Properties & Real Estate,H78,"[{'date': '2024-05-29', 'close': 4.50307108}, ..."
2,JCYC,1.068000e+10,1321700.0,6.49,2.223000e+10,0.399,neutral,neutral,strong_sell,-0.0146,...,0.46,1.26,0.9411,NaN,0.3245,0.3644,Consumer Cyclicals,Apparel & Luxury Goods,C07,"[{'date': '2024-05-29', 'close': 26.1299991608..."
3,CTDM,4.670000e+09,2551900.0,15.57,4.940000e+09,0.841,strong_sell,strong_sell,strong_sell,-0.0132,...,0.86,1.77,1.3871,0.0253,-0.4286,0.3070,Properties & Real Estate,Properties & Real Estate,C09,"[{'date': '2024-05-29', 'close': 5.9499998093}..."
4,SCIL,8.860000e+09,4134000.0,9.41,7.040000e+09,0.698,strong_sell,strong_sell,strong_buy,-0.0080,...,0.63,0.78,1.6486,0.0251,0.8571,0.1699,Infrastructures,Utilities,U96,"[{'date': '2024-05-29', 'close': 5.2199997902}..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,PAST,6.600000e+06,0.0,-17.39,1.109000e+07,0.000,sell,sell,sell,0.0000,...,0.93,1.58,0.5949,NaN,0.0000,-2.2801,Healthcare,Healthcare Equipment & Providers,UUK,"[{'date': '2024-05-29', 'close': 0.0500000007}..."
588,NIKS,1.963000e+07,19800.0,11.54,1.106000e+07,0.000,sell,strong_sell,neutral,0.0067,...,NaN,NaN,0.0000,NaN,0.0000,5.6346,Consumer Non-Cyclicals,Nondurable Household Products,NPL,"[{'date': '2024-05-29', 'close': 0.1620000005}..."
589,SHEF,3.632000e+07,0.0,9.56,2.759000e+07,0.000,strong_sell,strong_sell,buy,0.0000,...,3.04,3.59,0.0167,0.0459,0.0000,NaN,Industrials,Industrial Services,SGR,"[{'date': '2024-05-29', 'close': 0.200000003},..."
590,WINK,7.412000e+07,0.0,27.60,2.928000e+07,0.000,neutral,strong_buy,strong_buy,0.0000,...,NaN,NaN,0.0000,NaN,0.0000,NaN,Industrials,Industrial Services,WKS,"[{'date': '2024-05-29', 'close': 0.2549999952}..."


In [20]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import urllib.request

In [21]:
def GetGeneralData(country):
    if country == "sg":
        url ="https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=36&filter-domain=&page=0&page-size=1000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    elif country == "my":
        url = "https://api.investing.com/api/financialdata/assets/equitiesByCountry/default?fields-list=id%2Cname%2Csymbol%2CisCFD%2Chigh%2Clow%2Clast%2ClastPairDecimal%2Cchange%2CchangePercent%2Cvolume%2Ctime%2CisOpen%2Curl%2Cflag%2CcountryNameTranslated%2CexchangeId%2CperformanceDay%2CperformanceWeek%2CperformanceMonth%2CperformanceYtd%2CperformanceYear%2Cperformance3Year%2CtechnicalHour%2CtechnicalDay%2CtechnicalWeek%2CtechnicalMonth%2CavgVolume%2CfundamentalMarketCap%2CfundamentalRevenue%2CfundamentalRatio%2CfundamentalBeta%2CpairType&country-id=42&filter-domain=&page=0&page-size=2000&limit=0&include-additional-indices=false&include-major-indices=false&include-other-indices=false&include-primary-sectors=false&include-market-overview=false"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    data_from_api = None
    with urllib.request.urlopen(url) as response:
        html = response.read()

    data_from_api = json.loads(html)
    data_from_api = pd.DataFrame(data_from_api["data"])

    close_list = []
    for index, row in data_final.iterrows():
        now = datetime.now()
        prev_1month = datetime(now.year, now.month - 1 if now.month > 1 else 12, now.day).strftime("%Y-%m-%d")
        ticker = yf.Ticker(row["symbol"] + ".SI")
        yf_data = ticker.history(period="1mo").reset_index()
        curr = yf_data.iloc[-1]
        curr_date = curr["Date"].strftime("%Y-%m-%d")
        currency = row["currency"]
        data_final.loc[index, "volume"] = curr["Volume"]
        # try:
        #     if currency != "SGD":
        #         rate = float(data[currency]['SGD'])
        #         data_final.loc[index, "FundamentalMarketCap"] = ticker.info["marketCap"]*rate
        #     else:
        #         data_final.loc[index, "FundamentalMarketCap"] = ticker.info["marketCap"]
        # except:
        #     symbol = row["symbol"]
        #     print(f"error at {symbol} have no market cap")
        #     data_final.loc[index, "FundamentalMarketCap"] = np.nan
        if currency != "SGD":
            rate = float(data[currency]['SGD'])
            market_cap = row["FundamentalMarketCap"]*rate
            data_final.loc[index, "FundamentalMarketCap"] = market_cap
            curr_close = float(curr["Last"])*rate
        else:
            curr_close = float(curr["Last"])
        current_data = {
            "date" : curr_date,
            "close" : curr_close
        }
        close = [data for data in row["close"] if data["date"] > prev_1month]
        if current_data["date"] > close[-1]["date"]:
            close.append(current_data)
        close_list.append(close)
    data_final = data_final.assign(Last = close_list)

    # for i in range(10):
    #     response = requests.get(url, headers=headers)

    #     if response.status_code == 200:
    #         json_data = response.json()
    #         data_from_api = pd.DataFrame(json_data["data"])
    #         break
    return data_final

In [23]:
logging.basicConfig(filename="logs.log", level=logging.INFO)
data = GetGeneralData("sg")
links = data["Url"].tolist()
extension, failed_links = GetAdditionalData(links)
data_full = pd.merge(data, extension, on = "Url", how = "inner")
# Retry the failed links
n_try = 0
failed_links["links"] = [link.split("?")[0] if "?" in link else link for link in failed_links["links"]]
while len(failed_links["links"]) != 0 or n_try < 10:
    print(f"iterasi ke-{n_try+1}")
    if len(failed_links["links"]) == 0:
        break
    new_extension, failed_links = GetAdditionalData(failed_links["links"])
    n_try += 1
remaining = data[data["Url"].isin(failed_links["links"])]
remaining = remaining.assign(Url = [link.split("?")[0] if "?" in link else link for link in failed_links["links"]])
updated_extension = pd.merge(remaining, new_extension, on = "Url", how = "inner")
data_final = pd.concat([data_full[~data_full["Url"].isin(failed_links["links"])], updated_extension])
data_final = rename_and_convert(data_final)

error at overview page with link: /equities/ihh-healthcare-bhd?cid=955231
error at overview page with link: /equities/kop-ltd
error at overview page with link: /equities/sinostar-pec-holdings-ltd
error at ratios page with link: /equities/hengyang-petrochemical-logistics-lt?cid=991275
error at overview page with link: /equities/netlink


In [101]:
data_final.to_csv("data_full_sg_clean.csv")